In [1]:
import os
import cuvis
import cuvis_ai
import cuvis_ai.data
from PIL import Image as Img
import tqdm.notebook as tqdm
import matplotlib.pyplot as plt
from IPython.display import Image, display

In [2]:
# Download a publicly available dataset
base_path = "./cuvis_ai_video"
os.mkdir(base_path)
data_down = cuvis_ai.data.PublicDataSets()
data_down.download_dataset("Aquarium", download_path=base_path)

In [3]:
# Define number of parameters useful for understanding our classes
number_of_components = 6
number_of_classes = 4

In [5]:
# Define PCA with n components
pca = cuvis_ai.preprocessor.PCA(number_of_components)

In [8]:
# Define a GMM unsupervised classifier
gmm = cuvis_ai.unsupervised.GMM(number_of_classes)

In [10]:
# Define and construct graph
# This will throw an initialization warning "Unsatisfied dimensionality constraint"
# This is expected behavior
graph = cuvis_ai.pipeline.Graph("DemoGraph")
graph.add_base_node(pca)
graph.add_edge(pca, gmm)


In [ ]:
# Define unlabeled dataset
data = cuvis_ai.data.CuvisDataSet(base_path)

In [12]:
# Use first two images to fit the data
graph.fit(*data[0:1])

In [ ]:
# Generate gifs of session file classification
for i in tqdm.tqdm(range(len(data))):
    res_show, _, _ = graph.forward(*data[i:i+1])
    plt.figure()
    plt.imshow(res_show[0,:,:,:])  # Example plot
    plt.title('CUVIS AI Classification')
    filename = os.path.join(base_path
    , f"frame_{i}.png")
    plt.axis('off')
    plt.savefig(filename)
    plt.close()

In [14]:
# Save as an animated GIF

# Collect all the frames
frames = []
for i in range(len(data)):
    filename = os.path.join(base_path, f"frame_{i}.png")
    frames.append(Img.open(filename))
    
# Define save path
gif_path = os.path.join(base_path, "cuvis_fishtank_example_gmm.gif")
frames[0].save(
    gif_path,
    save_all=True,
    append_images=frames[1:],
    duration=50,  # Duration between frames in milliseconds
    loop=0  # Loop forever
)


In [ ]:
display(Image(filename=gif_path))